In [1]:
from statsmodels.stats.contingency_tables import mcnemar
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
#import seaborn as sns
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,GridSearchCV
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import calibration_curve,CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

### need a separate clean_and_featurize for Lasso and BERT, respectively

In [2]:
def clean_and_featurize(df,text_name='Report_Text_clean',BERT=True):
    """Something is happening here to clean the data...."""
    labeled = df
    #labeled = labeled[['Report_ID','Report_Text','MLS_mm']]
    
    #rename columns to relevant
    labeled = labeled.rename(columns={"Report_ID": "Report_Number"})
    
    # Keep only those with IMPRESSIONS
    #labeled = labeled.iloc[[x for x in range(labeled.shape[0]) if 'IMPRESSION:' in labeled.Report_Text.iloc[x]]]
    
    # replace whitespace with space ***************************
    labeled[text_name] = labeled['Report_Text'].apply(lambda text: ' '.join(text.split()))
    
    #REMOVE HEADER:
    labeled[text_name] = labeled[text_name].apply(lambda text: re.split('-'*78, text, 1)[-1])
    labeled[text_name] = labeled[text_name].apply(lambda text: re.split('HISTORY:', text, 1)[-1])
    labeled[text_name] = labeled[text_name].apply(lambda text: re.split('REPORT ', text, 1)[-1])
    labeled[text_name] = labeled[text_name].apply(lambda text: re.split('REPORT:', text, 1)[-1])
    
    #REMOVE FOOTER:
    # Remove footer parts
    labeled[text_name] = labeled[text_name].apply(lambda text: 
                                                       re.split('electronically signed by:', text, flags=re.IGNORECASE)[0])
    labeled[text_name] = labeled[text_name].apply(lambda text: 
                                                       ''.join(re.split('i, the teaching physician, have reviewed the images and agree with the report as written', text, flags=re.IGNORECASE)))
    labeled[text_name] = labeled[text_name].apply(lambda text: 
                                                       re.split('radiologists: signatures:', text, flags=re.IGNORECASE)[0])
    labeled[text_name] = labeled[text_name].apply(lambda text: 
                                                       re.split('providers: signatures:', text, flags=re.IGNORECASE)[0])
    labeled[text_name] = labeled[text_name].apply(lambda text: 
                                                       re.split('findings were discussed on', text, flags=re.IGNORECASE)[0])
    labeled[text_name] = labeled[text_name].apply(lambda text: 
                                                       re.split('this report was electronically signed by', text, flags=re.IGNORECASE)[0])

    # Remove reference texts =====
    labeled[text_name] = labeled[text_name].apply(lambda text: ''.join([x for i,x in enumerate(text.split('='*34)) if i != 1]))
    
    labeled[text_name] = labeled[text_name].apply(lambda text: text.lower())
    
    
    if not BERT: #if it's not BERT then go ahead and replace things with n-grams
    # Replace ngrams in Report_Text & IMPRESSION with their units
        for group in wordgroups:
            labeled[text_name] = labeled[text_name].apply(lambda text: text.replace(group, ''.join(group.split())))

    else:
        pass
    
    #labeled = labeled.drop_duplicates(subset=['Report_Number'])
    labeled = labeled.reset_index(drop=True)
    
    return labeled




def create_BOW_matrix_2(df1,df2,vectorizer='tfidf',doc_col_name='Report_Text_clean'):
    """Takes a DataFrame in which there is a column of documents, then parses through those
    texts in order to create a matrix in which the entries are the token counts per document.
    Returns this matrix as a DataFrame"""
    #initialize vectorizer
    if vectorizer=='count':
        vectorizer = CountVectorizer(min_df=2) #ignore if term appears in <cutoff % of documents
    else:
        vectorizer=TfidfVectorizer(min_df=2)
    #Obtain counts using the column name corresponding to the radiology reports
    vectorizer.fit(df1[doc_col_name])
    
    
    train_vocab=vectorizer.transform(df1[doc_col_name]).todense()
    test_vocab=vectorizer.transform(df2[doc_col_name]).todense()
    
    
    
    
    feature_names = vectorizer.get_feature_names()
    
    
    #Each one of the elements of the table 
    X_train = pd.DataFrame(train_vocab, columns = feature_names)
    X_test=pd.DataFrame(test_vocab,columns=feature_names)
    
    return X_train,X_test

def generate_mcnemar_table(df):
    """assume that the final column is the ground_truth and that you have two classifiers that are being
    compared"""
    ground_truth=df['true_label']
    
    lasso_corrects=[]
    bert_corrects=[]
    
    for _,row in df.iterrows():
        if row['lasso_pred']==row['true_label']:
            lasso_corrects.append(1)
        else:
            lasso_corrects.append(0)
            

    for _,row in df.iterrows():
        if row['bert_pred']==row['true_label']:
            bert_corrects.append(1)
        else:
            bert_corrects.append(0)    
    
    
    df2=pd.DataFrame(columns=['lasso_correct','bert_correct'])
    df2['lasso_correct']=lasso_corrects
    df2['bert_correct']=bert_corrects
    
    a=0
    b=0
    c=0
    d=0
    
    for _,row in df2.iterrows():
        if np.logical_and(row['lasso_correct']==1,row['bert_correct']==1):
            a+=1
        elif np.logical_and(row['lasso_correct']==1,row['bert_correct']==0):
            b+=1
        elif np.logical_and(row['lasso_correct']==0,row['bert_correct']==1):
            c+=1
        elif np.logical_and(row['lasso_correct']==0,row['bert_correct']==0):
            d+=1
    
    table=np.array([[a,b],[c,d]])
    
    return table
            
    
    

# Dry Run Script to Construct McNemar Table 

In [3]:
df = pd.read_csv('6.1.21 deidComplete REDCap Reports Non-duplicate Version 3.csv')
df.rename(columns={'report_text': 'Report_Text', 'mls_mm_v2': 'MLS_mm','record_id':'Report_ID','ecass_v2':'ECASS','edema_severity_report':'edema_severity'}, inplace=True)
wordgroups = list(pd.read_excel('wordgroups.xlsx',engine='openpyxl')['Word Groupings'])#now clean and featurize


df=df.dropna(subset=['ivh_v2'])
df['ivh_present']=df['ivh_v2'].apply(lambda x: 0 if x==1.0 else 1)
df['ECASS']=df['ECASS'].fillna(0)
df['severe_ecass']=df['ECASS'].apply(lambda x: 0 if x<3.0 else 1)
df['MLS_mm']=df['MLS_mm'].fillna(0)
df['MLS_presence']=df['MLS_mm'].apply(lambda x: 0 if x==0 else 1)
#df_edema = df[np.logical_or(df['MLS_presence']==0,df['MLS_presence']==1)]
df['edema_report']=df['edema_report'].fillna(0)
df['edema_report']=df['edema_report'].apply(lambda x: int(x))

input_spreadsheet=df

### Practice Training a LogReg and BERT side-by-side on same data split

### Main training loop:

In [11]:

num_trials=1

outcome='edema_report'

for trial in range(1,num_trials+1):
    print('Results for trial ',trial)
    
    input_spreadsheet=input_spreadsheet[np.logical_or(input_spreadsheet[outcome]==0,input_spreadsheet[outcome]==1)]
    
    
    ####MULTIPLE LEVELS OF TRAIN TEST SPLIT GOING ON HERE:
    #1: Overall train and test: 
    #reminder: y_overall_train and y_overall_test will be shared by both models
    X_overall_train,X_overall_test=train_test_split(input_spreadsheet,test_size=0.20)
    y_overall_train,y_overall_test=X_overall_train[outcome],X_overall_test[outcome]
    
    print('performed first split of the overall data')
    

    
    #2.derive BOW matrix for Lasso:
    #below we create BOW matrix from the n-gram converted reports
    X_lasso_train,X_lasso_test=create_BOW_matrix_2(clean_and_featurize(X_overall_train,BERT=False),
                                                  clean_and_featurize(X_overall_test,BERT=False)) 

    
#     print(X_lasso_train.shape)
#     print(X_lasso_test.shape)
#     #print(y_res.shape)
    
    print('performed featurization and split for Lasso')

    


    tuned_parameters =[{'tol': np.arange(0.0001, 0.1 + 0.01, 0.01)}]

        #initialize Logistic Regression with GridSearch over 'tolerance' parameter
    outcome_presence = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100, solver='liblinear'), 
                                tuned_parameters, cv = 10, scoring = 'roc_auc')

                #Calibrate the classifier with Platt scaling
    outcome_presence=CalibratedClassifierCV(outcome_presence)
    #SMOTE resampling
    outcome_presence.fit(X_lasso_train,y_overall_train)
    #derive the y predictions from lasso
    y_lasso_pred = outcome_presence.predict(X_lasso_test) #THIS PART IS ALL FINISHED NOW!
    
    print(len(y_lasso_pred))
    print(len(y_overall_test))



    
    
    
    #3. derive proper training inputs for BERT from X_overall_train
    #for simpletransformers library, must have 'text' and 'labels' columns
    renamed_df=X_overall_train.rename(columns={outcome:'labels'}) # <--------------------------------COME BACK TO THIS PART TOMORROW
    X_bert_train=clean_and_featurize(renamed_df,text_name='text')
    X_bert_train=X_bert_train[['text','labels']]
    
    
    #now derive BERT testing dataframe
    renamed_test_df=X_overall_test.rename(columns={outcome:'labels'})
    X_bert_test=clean_and_featurize(renamed_test_df,text_name='text')
    X_bert_test=X_bert_test[['text','labels']]
    
    print(X_bert_test.shape)
    print(X_bert_test.columns)
    
    
    
    #derive weights for training:
    num_zeros=X_bert_train[X_bert_train['labels']==0].shape[0]
    num_ones=X_bert_train[X_bert_train['labels']==1].shape[0]
    #derived weights
    weights=[1/num_zeros,1/num_ones]
    
    print(weights)
    
#     #initialize your model
#     model=ClassificationModel('bert','emilyalsentzer/Bio_ClinicalBERT',
#                          num_labels=2,use_cuda=True,weight=weights,
#                          args={'num_train_epochs':5,'output_dir':'mcnemar_outputs/','max_seq_length':512})
    
#     model.train_model(X_bert_train,num_training_epochs=10)
    
#     _,y_bert_pred,_=model.eval_model(X_bert_test) #<----NEED A BERT TESTING 
    
#     print(X_bert_train.shape)
#     print(y_overall_train.shape)
#     print(X_bert_train.columns)

#     #Now fill in the dataframe from which to derive McNemar cells
#     df=pd.DataFrame(columns=['lasso_pred','bert_pred','true_label'])
#     df['true_label']=y_overall_test
#     df['lasso_pred']=y_lasso_pred
#     df['bert_pred']=y_bert_pred
    
    
#     #now generate McNemar table and print statistic:
#     mcnemar_table=generate_mcnemar_table(df)
#     print(mcnemar(mcnemar_table))
    

    

Results for trial  1
performed first split of the overall data
performed featurization and split for Lasso
423
423
(423, 2)
Index(['text', 'labels'], dtype='object')
[0.0021929824561403508, 0.0008110300081103001]


In [6]:
def loop_mcnemar_calc(input_spreadsheet,outcome,num_trials=5):
    
    #only calculate on severe_ecass
    if outcome=='severe_ecass':
        input_spreadsheet=input_spreadsheet[input_spreadsheet['hem_conv_v2']==1]
    else:
        input_spreadsheet=input_spreadsheet



    for trial in range(1,num_trials+1):
        print('Results for trial ',trial)

        input_spreadsheet=input_spreadsheet[np.logical_or(input_spreadsheet[outcome]==0,input_spreadsheet[outcome]==1)]


        ####MULTIPLE LEVELS OF TRAIN TEST SPLIT GOING ON HERE:
        #1: Overall train and test: 
        #reminder: y_overall_train and y_overall_test will be shared by both models
        X_overall_train,X_overall_test=train_test_split(input_spreadsheet,test_size=0.20)
        y_overall_train,y_overall_test=X_overall_train[outcome],X_overall_test[outcome]

        print('performed first split of the overall data')



        #2.derive BOW matrix for Lasso:
        #below we create BOW matrix from the n-gram converted reports
        X_lasso_train,X_lasso_test=create_BOW_matrix_2(clean_and_featurize(X_overall_train,BERT=False),
                                                      clean_and_featurize(X_overall_test,BERT=False)) 


    #     print(X_lasso_train.shape)
    #     print(X_lasso_test.shape)
    #     #print(y_res.shape)

        print('performed featurization and split for Lasso')




        tuned_parameters =[{'tol': np.arange(0.0001, 0.1 + 0.01, 0.01)}]

            #initialize Logistic Regression with GridSearch over 'tolerance' parameter
        outcome_presence = GridSearchCV(LogisticRegression(penalty = 'l1',max_iter=1000, random_state=100, solver='liblinear'), 
                                    tuned_parameters, cv = 10, scoring = 'roc_auc')

                    #Calibrate the classifier with Platt scaling
        outcome_presence=CalibratedClassifierCV(outcome_presence)
        #SMOTE resampling
        outcome_presence.fit(X_lasso_train,y_overall_train)
        #derive the y predictions from lasso
        y_lasso_pred = outcome_presence.predict(X_lasso_test) #THIS PART IS ALL FINISHED NOW!

        print(len(y_lasso_pred))
        print(len(y_overall_test))



        #3. derive proper training inputs for BERT from X_overall_train
        #for simpletransformers library, must have 'text' and 'labels' columns
        renamed_df=X_overall_train.rename(columns={outcome:'labels'}) # <--------------------------------COME BACK TO THIS PART TOMORROW
        X_bert_train=clean_and_featurize(renamed_df,text_name='text')
        X_bert_train=X_bert_train[['text','labels']]


        #now derive BERT testing dataframe
        renamed_test_df=X_overall_test.rename(columns={outcome:'labels'})
        X_bert_test=clean_and_featurize(renamed_test_df,text_name='text')
        X_bert_test=X_bert_test[['text','labels']]

#         print(X_bert_test.shape)
#         print(X_bert_test.columns)



        #derive weights for training:
        num_zeros=X_bert_train[X_bert_train['labels']==0].shape[0]
        num_ones=X_bert_train[X_bert_train['labels']==1].shape[0]
        #derived weights
        weights=[1/num_zeros,1/num_ones]

#         print(weights)

#         initialize your model
        model=ClassificationModel('bert','emilyalsentzer/Bio_ClinicalBERT',
                             num_labels=2,use_cuda=True,weight=weights,
                             args={'num_train_epochs':1,'output_dir':'mcnemar_outputs/','overwrite_output_dir':True,'max_seq_length':512})

        model.train_model(X_bert_train,num_training_epochs=1)
        
        to_predict=[row['text'] for _,row in X_bert_test.iterrows()]

        y_bert_pred,_=model.predict(to_predict) #<----NEED A BERT TESTING 

#         print(X_bert_train.shape)
#         print(y_overall_train.shape)
#         print(X_bert_train.columns)
#         print(X_lasso_train.columns)


        #Now fill in the dataframe from which to derive McNemar cells
        df=pd.DataFrame(columns=['lasso_pred','bert_pred','true_label'])
        df['true_label']=y_overall_test
        df['lasso_pred']=y_lasso_pred
        df['bert_pred']=y_bert_pred
        print(df)


        #now generate McNemar table and print statistic:
        mcnemar_table=generate_mcnemar_table(df)
        print(mcnemar_table)
        print(mcnemar(mcnemar_table))




In [7]:
#for out in ['edema_report','MLS_presence','hem_conv_v2','severe_ecass']:
for out in ['edema_report','severe_ecass']:    
    loop_mcnemar_calc(input_spreadsheet,out,num_trials=1)
    

Results for trial  1
performed first split of the overall data
performed featurization and split for Lasso
423
423


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

  0%|          | 0/1691 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/212 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

      lasso_pred  bert_pred  true_label
834            0          1           0
168            1          1           1
2008           0          1           0
1426           0          1           0
80             0          1           0
...          ...        ...         ...
1361           0          1           0
1941           1          1           0
1935           1          1           0
888            0          1           1
455            0          1           0

[164 rows x 3 columns]
[[45 76]
 [24 19]]
pvalue      1.810002621302925e-07
statistic   24.0
